下面根据customized c 模型，做一些调整。以期望达到更快的收敛。

1. 加两个卷积层
2. 加3个dense层
3. 将卷积的核，从5*5 改为3*3


## error
ValueError: Dimension size must be evenly divisible by 16384 but is 102400 for 'Reshape_1' (op: 'Reshape') with input shapes: [100,4,4,64], [2] and with input tensors computed as partial shapes: input[1] = [?,16384].


shape of conv4:  Tensor("conv2d_4/Relu:0", shape=(100, 8, 8, 64), dtype=float32) ; and shape of pool4 is:  Tensor("max_pooling2d_4/MaxPool:0", shape=(100, 4, 4, 64), dtype=float32)

上一层，与下一层的参数关系？

```
# Convolutional Layer #2 and Pooling Layer #2
  conv4 = tf.layers.conv2d(
      inputs=pool3,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    
    
  # Dense Layer
  print ('shape of conv4: ', conv4, '; and shape of pool4 is: ', pool4)

  pool4_flat = tf.reshape(pool4, [-1, 16 * 16 * 64])
  dense1 = tf.layers.dense(inputs=pool4_flat, units=1024, activation=tf.nn.relu)
```

将这个：  pool4_flat = tf.reshape(pool4, [-1, 16 * 16 * 64])
改为和上一层的shape一样：
  pool4_flat = tf.reshape(pool4, [-1, 4 * 4 * 64])
就不再报错了。


INFO:tensorflow:Starting evaluation at 2018-04-27-11:18:50
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-d/model.ckpt-100
INFO:tensorflow:Finished evaluation at 2018-04-27-11:18:57
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.0271493, global_step = 100, loss = 3.61113

Test Accuracy: 2.714932%

INFO:tensorflow:Starting evaluation at 2018-04-27-11:41:36
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-d/model.ckpt-1100
INFO:tensorflow:Finished evaluation at 2018-04-27-11:41:44
INFO:tensorflow:Saving dict for global step 1100: accuracy = 0.0271493, global_step = 1100, loss = 3.61091

Test Accuracy: 2.714932%


INFO:tensorflow:Starting evaluation at 2018-04-27-12:25:38
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-d/model.ckpt-2100
INFO:tensorflow:Finished evaluation at 2018-04-27-12:25:45
INFO:tensorflow:Saving dict for global step 2100: accuracy = 0.0285068, global_step = 2100, loss = 3.61116

Test Accuracy: 2.850679%

下面的模型虽然不出错，但是感觉有问题：
INFO:tensorflow:Starting evaluation at 2018-04-28-02:09:40
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-d/model.ckpt-5100
INFO:tensorflow:Finished evaluation at 2018-04-28-02:09:48
INFO:tensorflow:Saving dict for global step 5100: accuracy = 0.0380091, global_step = 5100, loss = 3.61053

Test Accuracy: 3.800905%


感觉这个层数组合，是有说法的。

In [8]:
import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
  print ('shape of input_layer: ', input_layer)
  # with batch_size =100, shape should be: [100, 28, 28, 1]
  # shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  print ('shape of conv1: ', conv1)
  # shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
    
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  print ('shape of conv2: ', conv2, '; and shape of pool2 is: ', pool2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv4 = tf.layers.conv2d(
      inputs=pool3,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    
    
  # Dense Layer
  print ('shape of conv4: ', conv4, '; and shape of pool4 is: ', pool4)

  pool4_flat = tf.reshape(pool4, [-1, 4 * 4 * 64])
  dense1 = tf.layers.dense(inputs=pool4_flat, units=1024, activation=tf.nn.relu)
  dense2 = tf.layers.dense(inputs=dense1, units=1024, activation=tf.nn.relu)
  dense3 = tf.layers.dense(inputs=dense2, units=1024, activation=tf.nn.relu)
  dense4 = tf.layers.dense(inputs=dense3, units=1024, activation=tf.nn.relu)

  dropout = tf.layers.dropout(
      inputs=dense4, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=CHARSET_SIZE)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


class DataSetLoader:
    def __init__(self, data_dir):
        # Set FLAGS.charset_size to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        print ('self.images: ', self.images[0].dtype)
        self.images = np.float32(self.images)
        print ('self.images: ', self.images[0].dtype)

        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

    
train_data = DataSetLoader(data_dir='../data/train_/')
print ('Train data loaded ...')


# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]



# Build CNN with customized function ...
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
   model_dir='../dfs/checkpoint/customized_model-d')

    

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(train_data)[0]},
    y=input(train_data)[1],
    num_epochs=None,
    batch_size=100,
    shuffle=True
)
print ('Begin to train ...')

classifier.train(input_fn=train_input_fn, steps=3000)
print ('Train done ...')

test_data = DataSetLoader(data_dir='../data/test_/')
print ('Test data loaded ...')

# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

Now processing path:  ../data/train_/00037
INFO:tensorflow:global_step/sec: 0.831184
INFO:tensorflow:loss = 3.61759, step = 2201 (120.313 sec)
INFO:tensorflow:global_step/sec: 0.842564
INFO:tensorflow:loss = 3.61024, step = 2301 (118.685 sec)
INFO:tensorflow:global_step/sec: 0.845224
INFO:tensorflow:loss = 3.60715, step = 2401 (118.312 sec)
INFO:tensorflow:global_step/sec: 0.846627
INFO:tensorflow:loss = 3.61285, step = 2501 (118.117 sec)
INFO:tensorflow:global_step/sec: 0.851396
INFO:tensorflow:loss = 3.61231, step = 2601 (117.454 sec)
INFO:tensorflow:Saving checkpoints for 2607 into ../dfs/checkpoint/customized_model-d/model.ckpt.
INFO:tensorflow:global_step/sec: 0.849203
INFO:tensorflow:loss = 3.6086, step = 2701 (117.757 sec)
INFO:tensorflow:global_step/sec: 0.828508
INFO:tensorflow:loss = 3.60935, step = 2801 (120.698 sec)
INFO:tensorflow:global_step/sec: 0.845095
INFO:tensorflow:loss = 3.61098, step = 2901 (118.333 sec)
INFO:tensorflow:global_step/sec: 0.858014
INFO:tensorflow:lo

下面是直接测试版：



In [9]:
# from: https://www.kaggle.com/jeffcarp/example-save-and-load-a-tensorflow-model

import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

from tensorflow.contrib.learn.python.learn.estimators import run_config
from tensorflow.contrib.training.python.training import hparam

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
  print ('shape of input_layer: ', input_layer)
  # with batch_size =100, shape should be: [100, 28, 28, 1]
  # shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  print ('shape of conv1: ', conv1)
  # shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
    
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  print ('shape of conv2: ', conv2, '; and shape of pool2 is: ', pool2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv4 = tf.layers.conv2d(
      inputs=pool3,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    
    
  # Dense Layer
  print ('shape of conv4: ', conv4, '; and shape of pool4 is: ', pool4)

  pool4_flat = tf.reshape(pool4, [-1, 4 * 4 * 64])
  dense1 = tf.layers.dense(inputs=pool4_flat, units=1024, activation=tf.nn.relu)
  dense2 = tf.layers.dense(inputs=dense1, units=1024, activation=tf.nn.relu)
  dense3 = tf.layers.dense(inputs=dense2, units=1024, activation=tf.nn.relu)
  dense4 = tf.layers.dense(inputs=dense3, units=1024, activation=tf.nn.relu)

  dropout = tf.layers.dropout(
      inputs=dense4, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=CHARSET_SIZE)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        self.images = np.float32(self.images)
        # or else: TypeError: Value passed to parameter 'input' has DataType float64 not in list of allowed values: float16, float32

        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

# Build CNN with customized function ...
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
   model_dir='../dfs/checkpoint/customized_model-d')



# MODEL_DIR = "../dfs/checkpoint/customized_model-c"
# model_from_checkpoint = make_estimator(MODEL_DIR)

test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/customized_model-d', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Now processing path:  ../data/test_/00037
shape of input_layer:  Tensor("Reshape:0", shape=(?, 64, 64, 1), dtype=float32)
shape of conv1:  Tensor("conv2d/Relu:0", shape=(?, 64, 64, 32), dtype=float32)
shape of conv2:  Tensor("conv2d_2/Relu:0", shape=(?, 32, 32, 64), dtype=float32) ; and shape of pool2 is:  Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
shape of conv4:  Tensor("conv2d_4/Relu:0", shape=(?, 8, 8, 64), dtype=float32) ; and shape of pool4 is:  Tensor("max_pooling2d_4/MaxPool:0", shape=(?, 4, 4, 64), dtype=float32)
INFO:tensorflow:Starting evaluation at 2018-04-28-02:09:40
INFO:tensorflow:Restor